<a href="https://colab.research.google.com/github/FrancescaSrc/AIprojects/blob/master/Tv_script_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TV Script Generation

In this project, I have generated a [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs using  part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network trained generates a new ,"fake" TV script, based on patterns it recognizes in the given training data.

## The Data

The original data is can be found in the `Seinfeld_Scripts.txt` . If you are trying out the script, place it in the right directory.

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data
  

# load in data
import os
import pickle
import torch
data_dir = 'drive/My Drive/Colab Notebooks/../data/Seinfeld_Scripts.txt'
text = load_data(data_dir)

In [0]:

# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data


In [0]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
##  Pre-processing Functions
The first thing to do to any dataset is pre-processing.  
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding: first transform the words to ids and create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`


In [0]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab




tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation


Function `token_lookup`  returns a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token_dic={
        '.' : '||PERIOD||',
        ',' : '||COMMA||',
        '?' : '||QUOTATION_MARK||',
        ';' : '||SEMICOLON||',
        '!' : '||EXCLAMATION_MARK||',
        '?' : '||QUESTION_MARK||',
        '(' : '||LEFT_PAREN||',
        ')' : '||RIGHT_PAREN||',
        '-' : '||DASH||',
        '\n' : '||RETURN||',
        '"' : '||QUOTATION_MARK||'
              
    }
        
    return token_dic

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. Helper file was given.

In [0]:

# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
The preprocessed data has been saved to disk.

In [0]:

import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [0]:

import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

### Creata a Dataloader and Batching
Implementation of the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.




In [0]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    batch_size_total = batch_size * sequence_length
    # total number of batches we can make
    n_batches = len(words)//batch_size_total
    x, y = [], []
    for idx in range(0, len(words), batch_size):
        batch = words[idx:idx+batch_size]
        #print(batch)
        for ii in range(len(batch)-sequence_length):
            batch_x = batch[ii:sequence_length+ii]
            #print("feature: ",batch_x)
            batch_y = batch[len(batch_x)+ii]
            #print("target: ",batch_y)
            y.extend([batch_y])
            x.extend([batch_x]*len([batch_y]))
       
    #print('features', x)
    #print('targets', y)
    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size, shuffle=True)
    # return a dataloader
    return data_loader


# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


In [0]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[42, 43, 44, 45, 46],
        [ 2,  3,  4,  5,  6],
        [14, 15, 16, 17, 18],
        [ 0,  1,  2,  3,  4],
        [44, 45, 46, 47, 48],
        [41, 42, 43, 44, 45],
        [32, 33, 34, 35, 36],
        [23, 24, 25, 26, 27],
        [21, 22, 23, 24, 25],
        [24, 25, 26, 27, 28]])

torch.Size([10])
tensor([47,  7, 19,  5, 49, 46, 37, 28, 26, 29])


---
## Build the Neural Network

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.4):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        # set class variables
        super(RNN, self).__init__()
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        
        
         # define model layers
        
        # embedding and LSTM layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)

        
        
      
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        
        # embeddings and lstm_out

        embeds = self.embed(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        output = self.dropout(lstm_out)
        output = self.fc(output)

        
        # reshape into (batch_size, seq_length, output_size)
        output = output.view(batch_size, -1, self.output_size)
        # get last batch 
        out = output[:, -1]

        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        weight = next(self.parameters()).data
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden


tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation anda move your data to the  GPU device.**

In [0]:
from torch.optim import lr_scheduler
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if (train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
        rnn.cuda()
    
    # Representing Memory ##
    # make a new variable for hidden and detach the hidden state from its history
    # this way, we don't backpropagate through the entire history
    hidden = tuple([each.data for each in hidden])
    # perform backpropagation and optimization  
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    nn.utils.clip_grad_norm_(rnn.parameters(), 3)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
   
    

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training


In [0]:


def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters
The neural network is set to work with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.


In [0]:
# Data params
# Sequence Length
sequence_length = 40  # of words in a sequence
# Batch Size
batch_size = 60

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 25
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int.keys())
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 400
# Hidden Dimension
hidden_dim = 700
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 1000

In [0]:
def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)


def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

In [0]:


# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
save_model('drive/My Drive/Colab Notebooks/../trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 25 epoch(s)...
Epoch:    1/25    Loss: 5.252101881742478

Epoch:    1/25    Loss: 4.832055360794067

Epoch:    1/25    Loss: 4.668700470924377

Epoch:    1/25    Loss: 4.586288829326629

Epoch:    2/25    Loss: 4.468339825334724

Epoch:    2/25    Loss: 4.3520312111377715

Epoch:    2/25    Loss: 4.313439683198929

Epoch:    2/25    Loss: 4.316178456544876

Epoch:    3/25    Loss: 4.2098778218091875

Epoch:    3/25    Loss: 4.16459723329544

Epoch:    3/25    Loss: 4.155525993347168

Epoch:    3/25    Loss: 4.176868861436843

Epoch:    4/25    Loss: 4.081175246487366

Epoch:    4/25    Loss: 4.045531676769256

Epoch:    4/25    Loss: 4.040482199907303

Epoch:    4/25    Loss: 4.058029629707336

Epoch:    5/25    Loss: 4.0005541735143995

Epoch:    5/25    Loss: 3.931645712852478

Epoch:    5/25    Loss: 3.9380845232009887

Epoch:    5/25    Loss: 3.9749956018924713

Epoch:    6/25    Loss: 3.9109233550742606

Epoch:    6/25    Loss: 3.875240173101425

Epoch:    6/25    Los

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


---
# Checkpoint

Saving the model as `trained_rnn`, resume by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [0]:

import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')


In [0]:
trained_rnn.to("cpu")
train_on_gpu = False

## Generate TV Script
With the network trained and saved, generate a new, "fake" Seinfeld TV script in this section.

### Generate Text function
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. 

In [0]:
#project given function

import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script


In [0]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry:

jerry: well, i think i don't have to be able to do that.(walks over to the phone with his hands, and knocks them to leave)...

kramer: oh, i can't believe you, you know the ones that means that the thing was that the best thing that i am, and then the only way to be a lot of money, but i don't want it to get out of the office. you can't be able to see him again.

elaine: oh, i don't know.

kramer: well, i'm gonna get a lot of a bad impression.

elaine: i can't believe this.

jerry: what do you mean?

george: yeah, but you don't know what happened in the world. i can't believe it.

jerry: well, i'm going to the tony's.

kramer:(to jerry, upset) i don't know.

jerry: oh, yeah.

elaine:(looking at jerry) you have a message?

jerry: i know i was thinking about the other night, i was a little bit higher.

jerry: yeah, well i don't know.

george: what?(kramer enters)

jerry: hey.

newman: what are you looking at me?

kramer: oh, yeah, yeah. i don't know, i just came back.

jerry:(to 

#### Save the script

Once you have a script that you like (or find interesting), save it to a text file!

In [0]:
# save script to a text file
f =  open("drive/My Drive/Colab Notebooks/../generated_script_1.txt","w")
f.write(generated_script)
f.close()

**Experience and skills developed with this project**

This project has been a great resource to study the working of RNN in great detail. The generation of text is not a simple task!
Skills I have delevoped:

- NLP skills: create dictionary, encode text and create embedings layers
- Deep learning: build and apply an RNN for text generation, create batches and embedding layers, understand LSTM and GRU structure
- Course knowledge: RNNs, LSTMs, Attention model in Sequence to sequence models 